In [1]:
import IPython
import itertools as it
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

def gen_board(n, n_colors, p_alive):
    values = tuple(range(n_colors + 1))
    probs = (1 - p_alive,) + tuple(p_alive / n_colors for v in range(n_colors))
    return np.random.choice(values, n*n, p=probs).reshape(n, n)

def get_neighs(x, y, board_size, surf):
    if surf == 'torus':
        x_range = tuple(i % board_size for i in range(x-1, x+2))
        y_range = tuple(j % board_size for j in range(y-1, y+2))
    elif surf == 'plane':
        x_range = tuple(range(max(0, x-1), min(x+2, board_size)))
        y_range = tuple(range(max(0, y-1), min(y+2, board_size)))
    neighs = (e for e in it.product(x_range, y_range) if e != (x, y))
    return neighs

def update_board(board, n_colors, surf, rules):
    board_size = board.shape[0]
    new_board = np.zeros((board_size, board_size)).astype(int)
    for e in it.product(range(board_size), repeat=2):
        i, j = e[0], e[1]
        status = board[i, j]
        neighs = get_neighs(i, j, board_size, surf)
        neigh_vals = tuple(board[n[0], n[1]] for n in neighs)
        non_zero_vals = tuple(n for n in neigh_vals if n != 0)
        n_alive = len(non_zero_vals)
        # live next gen according to rule set
        if n_alive in rule_sets[rules][int(status > 0)]:
            counts = {n: non_zero_vals.count(n) for n in non_zero_vals}
            n_max = {n for n in non_zero_vals if counts[n] == max(counts.values())}
            # criteria for colour selection
            if len(n_max) == 1:
                new_color = n_max.pop()
            elif board[i, j] > 0:
                new_color = board[i, j]
            elif n_max:
                viable = [n for n in tuple(range(1, n_colors+1)) if n not in n_max]
                new_color = np.random.choice(viable)
            else:
                new_color = np.random.choice(range(1, n_colors+1))
            new_board[i, j] = new_color
    return new_board

def run_life(board=20, n_frames=60, n_colors=4, p_alive=0.05, surf='torus', rules='classic'):
    """
    Makes list of epochs of a cellular automaton.
    
    Keyword arguments:
    board -- either a pre-defined n*n numpy array or the side of a new n*n zeros board
    n_frames -- number of epochs in the animation
    n_colors -- number of cell colours available
    p_alive -- probability of a cell being born alive, if starting from a new board
    surf -- to determine cell adjacency, options are 'torus' and 'plane'
    rules -- rule set to determine which cells will live next turn, selected from rule_sets
    """
    board = gen_board(board, n_colors, p_alive) if isinstance(board, int) else board
    epochs = [board]
    for i in range(n_frames):
        board = update_board(board, n_colors, surf, rules)
        epochs.append(board)
    return epochs

def center_pattern(pattern, board_size, row_adj=0, col_adj=0):
    """
    Centers a pattern on a board.
    
    Keyword arguments:
    pattern -- the pattern to center, an arbitrary 2d array
    board_size -- the size of the board to generate
    row_adj -- row displacement needed due to latter epochs
    col_adj -- col displacement needed due to latter epochs
    """
    board = gen_board(board_size, n_colors=1, p_alive=0)
    offset = math.floor(board_size / 2)
    row_off = offset - math.ceil(max([s[0] / 2 for s in pattern])) + row_adj
    col_off = offset - math.ceil(max([s[1] / 2 for s in pattern])) + col_adj
    for square in pattern:
        colour = square[2] if len(square) == 3 else 1
        board[square[0] + row_off, square[1] + col_off] = colour
    return board

def discrete_cmap(N, base_cmap=None):
    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

def make_ani(board=20, n_frames=60, n_colors=4, p_alive=0.05, surf='torus', rules='classic', n_first=3):
    """
    Make animation of a cellular automaton.
    
    Keyword arguments:
    board -- either a pre-defined n*n numpy array or the side of a new n*n zeros board
    n_frames -- number of epochs in the animation
    n_colors -- number of cell colours available
    p_alive -- probability of a cell being born alive, if starting from a new board
    surf -- to determine cell adjacency, options are 'torus' and 'plane'
    rules -- rule set to determine which cells will live next turn, selected from rule_sets
    n_first -- number of frames for first epoch, e.g. 0 for random start and 3 for starting pattern
    """
    life_run = run_life(board=board,
                        n_frames=n_frames,
                        n_colors=n_colors,
                        p_alive=p_alive,
                        surf=surf,
                        rules=rules)
    life_run = [life_run[0]]*n_first + life_run[1:]
    fig = plt.figure(figsize=(8, 8), tight_layout=True)
    cmap = discrete_cmap(n_colors+1, 'CMRmap')
    def init():
        data = life_run[0]
        hm = sns.heatmap(data, vmin=0, vmax=n_colors,
                         square=True, cbar=False, cmap=cmap,
                         xticklabels=False, yticklabels=False)
        return hm
    def animate(i):
        data = life_run[i]
        hm = sns.heatmap(data, vmin=0, vmax=n_colors,
                         square=True, cbar=False, cmap=cmap, 
                         xticklabels=False, yticklabels=False)
        return hm
    ani = mpl.animation.FuncAnimation(fig, animate, frames=n_frames+n_first-1)
    movie = IPython.display.HTML(ani.to_html5_video())
    plt.close()
    return movie

rule_sets = {
    '2_by_2': ((3,6), (1,2,5)),
    '34_life': ((3,4), (3,4)),
    'anneal': ((4,6,7,8), (3,5,6,7,8)),
    'b25_s4': ((2,5), (4,)),
    'classic': ((3,), (2,3)),
    'day_and_night': ((3,6,7,8), (3,4,6,7,8)),
    'diamoeba': ((3,5,6,7,8), (5,6,7,8)),
    'dry_life': ((3,7), (2,3)),
    'flock': ((3,), (1,2)),
    'fredkin': ((1,3,5,7), (0,2,4,6,8)),
    'high_life': ((3,6), (2,3)),
    'live_free_or_die': ((2,), (0,)),
    'maze': ((3,), (1,2,3,4,5)),
    'mazectric': ((3,), (1,2,3,4)),
    'morley': ((3,6,8), (2,4,5)),
    'move': ((3,6,8), (2,4,5)),
    'no_death': ((3,), (0,1,2,3,4,5,6,7,8)),
    'pedestrian_life': ((3,8), (2,3)),
    'replicator': ((1,3,5,7), (1,3,5,7)),
    'seeds': ((2,), (-1,)),
}

In [2]:
make_ani(board=50, n_frames=60, n_colors=1, p_alive=0.1, surf='torus', rules='maze', n_first=0)

In [11]:
ships = (
    (0,5,2),(0,6,2),
    (3,0,1),
    (4,0,1),
    (5,9,1),
    (6,9,1),
    (9,3,2),(9,4,2),
)

board = center_pattern(ships, 120)
make_ani(board, n_frames=100, n_colors=3, surf='plane', rules='seeds')

In [4]:
simple_ships = (
     (0,4,1),(0,5,1),
    (1,3,1),  (1,6,1),
    (3,1,2),(3,8,2),
    (4,0,2),(4,9,2),
    (5,0,2),(5,9,2),
    (6,1,2),(6,8,2),
    (8,3,1),(8,6,1),
    (9,4,1),(9,5,1),
)

board = center_pattern(simple_ships, 60)
make_ani(board, n_frames=100, n_colors=3, surf='torus', rules='seeds')

In [5]:
Y_shape = (
    (0,0),      (0,2),
    (1,0),(1,1),(1,2),
          (2,1)
)

board = center_pattern(Y_shape, 49)
make_ani(board, n_frames=200, n_colors=4)

In [6]:
boxes = (
    (0,0,2),(0,1,3),(0,2,2),
    (1,0,3),        (1,2,3),
            (2,1,4),        (2,3,4),
            (3,1,1),(3,2,4),(3,3,1),
)

board = center_pattern(boxes, 26)
make_ani(board, n_frames=100, n_colors=4)

In [7]:
canes = (
    (0,0,3),(0,1,2),(0,2,3),
    (1,0,2),        (1,2,2),
    (2,0,3),                (2,3,4),
            (3,1,1),        (3,3,1),
            (4,1,4),(4,2,1),(4,3,4),
)

board = center_pattern(canes, 25)
make_ani(board, n_frames=70, n_colors=4)

In [8]:
long_canes = (
    (0,0,3),(0,1,2),(0,2,3),
    (1,0,2),        (1,2,2),
    (2,0,3),                (2,3,1),
    (3,0,2),                (3,3,4),
            (4,1,1),        (4,3,1),
            (5,1,4),(5,2,1),(5,3,4),
)

board = center_pattern(long_canes, 12)
make_ani(board, n_frames=10, n_colors=4)

In [9]:
die_hard = (
                         (0,6),
    (1,0),(1,1),
          (2,1),   (2,5),(2,6),(2,7),
)

board = center_pattern(die_hard, 25, row_adj=-6, col_adj=1)
make_ani(board, n_frames=135, n_colors=2)

In [10]:
queen_bee = (
    (0,0),
    (1,0),      (1,2),
          (2,1),      (2,3),
          (3,1),            (3,4),
          (4,1),      (4,3),
    (5,0),      (5,2),
    (6,0),
)

board = center_pattern(queen_bee, 47, col_adj=-7)
make_ani(board, n_frames=200, n_colors=3)